In [1]:
# Get all data paths
import os
paths = {0: [], 1: [], 2: []}
folder_names = ["Cloud", "Edge", "Good"]
for i, folder_name in enumerate(folder_names):
    folder_path = os.path.join("..\data", folder_name)
    for file_name in os.listdir(folder_path):
        paths[i].append(os.path.join(folder_path, file_name))
print(len(paths[0]), len(paths[1]), len(paths[2]))
print(paths[0][0])

90 97 118
..\data\Cloud\img_msec_1608603767961_2_half_1.png


In [2]:
import cv2
import numpy as np
from openvino.preprocess import PrePostProcessor, ResizeAlgorithm, ColorFormat
from openvino.runtime import Core, Layout, Type


In [3]:
model_path = 'model_mobilenet_v2.onnx'
device_name = 'MYRIAD'

In [4]:
# Initialize openvino runtime core
print('Creating OpenVINO Runtime Core')
core = Core()

Creating OpenVINO Runtime Core


In [26]:
# Read ONNX model
print(f'Reading the model: {model_path}')
# (.xml and .bin files) or (.onnx file)
model = core.read_model(model_path)

Reading the model: model_mobilenet_v2.onnx


In [6]:
# Read input image
image_paths = paths[0] + paths[1] + paths[2]
# images = [cv2.imread(path) for path in image_paths]
# count = 0
# for i, image in enumerate(images):
#     if image.shape != (1942, 1024, 3):
#         print(f"Image {i} has shape {image.shape}")
#         count += 1
# print(f"Total {count} images have wrong shape")
images = []
print("Reading images")
for path in image_paths:
    image = cv2.imread(path)
    # make grayscale
    image = cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR)
    if image.shape == (1942, 1024, 3):
        images.append(image)
    else:
        images.append(cv2.resize(image, (1024, 1942)))
print("Putting images into numpy array")
complete_tensor = np.array(images)

Reading images
Putting images into numpy array


In [7]:
# Define batch size
batch_size = 1
example_input = complete_tensor[:batch_size]

In [27]:
# Preprocess input image
ppp = PrePostProcessor(model)

_, h, w, _ = example_input.shape

# 1) Set input tensor information:
# - input() provides information about a single model input
# - reuse precision and shape from already available `input_tensor`
# - layout of data is 'NHWC'
ppp.input().tensor() \
    .set_shape(example_input.shape) \
    .set_element_type(Type.u8) \
    .set_color_format(ColorFormat.BGR) \
    .set_layout(Layout('NHWC'))  # noqa: ECE001, N400

dataset_mean = [0.2391, 0.4028, 0.4096]
dataset_std = [0.2312, 0.3223, 0.3203]
# 2) Adding explicit preprocessing steps:
# - apply linear resize from tensor spatial dims to model spatial dims
ppp.input().preprocess()\
    .resize(ResizeAlgorithm.RESIZE_LINEAR)\
    .convert_element_type(Type.f32)\
    .convert_color(ColorFormat.RGB)\
    .scale(255)\
    .mean(dataset_mean)\
    .scale(dataset_std)

# transform = transforms.Compose([v2.ToImage(),
#                                 v2.Resize((int(256), int(256))),
#                                 v2.RandomHorizontalFlip(p=0.5),
#                                 v2.RandomVerticalFlip(p=0.5),
#                                 v2.ToDtype(torch.float32, scale=True),
#                                 v2.Grayscale(num_output_channels=3),
#                                 v2.Normalize(dataset_mean,dataset_std)
#                                 ])
# 3) Here we suppose model has 'NCHW' layout for input
ppp.input().model().set_layout(Layout('NCHW'))

# 4) Set output tensor information:
# - precision of tensor is supposed to be 'f32'
ppp.output().tensor().set_element_type(Type.f32)

# 5) Apply preprocessing modifying the original 'model'
processed_model = ppp.build()

In [28]:
import time
# Load the model into the device
print(f'Loading the model to the {device_name} device')
start = time.time()
compiled_model = core.compile_model(processed_model, device_name)
end = time.time()
print(f"Total time: {end - start:.3f}s")

Loading the model to the MYRIAD device
Total time: 6.596s


In [29]:
# Create infer request
print('Starting inference in synchronous mode')
start = time.time()
# for i in range(iterations):
#     results = compiled_model.infer_new_request({0: example_input})
results = []
for i, x in enumerate(complete_tensor):
    y = compiled_model.infer_new_request({0: np.expand_dims(x, 0)})
    predictions = next(iter(y.values()))
    probs = predictions.reshape(-1)
    results.append(probs)
    if i % 10 == 0:
        print(f"\r {i} / {len(complete_tensor)}", end="")
end = time.time()
print(f"\r{len(complete_tensor)} / {len(complete_tensor)}")
total = end - start
average = total / len(complete_tensor)
print(f"Total time: {total:.3f}s")
print(f"Average time: {average:.3f}s")

Starting inference in synchronous mode
305 / 3055
Total time: 28.840s
Average time: 0.095s


In [30]:
# Get accuracy
results = np.array(results)
actual = np.array([0] * len(paths[0]) + [1] * len(paths[1]) + [2] * len(paths[2]))
accuracy = np.mean(np.argmax(results, axis=1) == actual)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.843


In [31]:
# Comparison to torch model
import torchvision.models as models
import torch
import torch.nn.functional as F

model_path = 'model_mobilenet_v2.pth'
model = models.mobilenet_v2()
model.load_state_dict(torch.load(model_path))

def test(model, test_loader, loss_fn, num_runs=5, device="cpu"):
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0

    for run in range(num_runs):
        num_correct = 0
        num_examples = 0
        count = 0
        for batch in test_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)

            with torch.no_grad():
                output = model(inputs)
                loss = loss_fn(output, targets)
                total_loss += loss.item() * inputs.size(0)

            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
            count += 1
            if count % 10 == 0:
                print(f"\r{count} / {len(test_loader)}: {targets} {torch.max(F.softmax(output, dim=1), dim=1)[1]} {correct}", end="")
        print(f"\r{len(test_loader)} / {len(test_loader)}")
        accuracy = num_correct / num_examples
        total_accuracy += accuracy

    average_loss = total_loss / (num_runs * len(test_loader.dataset))
    average_accuracy = total_accuracy / num_runs

    print('Average Test Loss: {:.2f}, Average Test Accuracy: {:.2f}'.format(average_loss, average_accuracy))

In [33]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.transforms import v2

path_data=os.path.join("..", "data")
dataset_mean = [0.2391, 0.4028, 0.4096]
dataset_std = [0.2312, 0.3223, 0.3203]

transform = transforms.Compose([v2.ToImage(),
                                v2.Resize((int(256), int(256))),
                                v2.RandomHorizontalFlip(p=0.5),
                                v2.RandomVerticalFlip(p=0.5),
                                v2.ToDtype(torch.float32, scale=True),
                                v2.Grayscale(num_output_channels=3),
                                v2.Normalize(dataset_mean,dataset_std)
                                ])

dataset = datasets.ImageFolder(root=path_data,
                                 transform=transform)
test_loader = DataLoader(dataset, batch_size=4, pin_memory=False, shuffle=True)
test(model, test_loader, torch.nn.CrossEntropyLoss(), num_runs=1, device="cpu")

C:\Users\siddd\anaconda3\envs\OMICRON2\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


77 / 77: tensor([0, 0, 0, 2]) tensor([0, 0, 0, 2]) tensor([True, True, True, True])ue])
Average Test Loss: 0.33, Average Test Accuracy: 0.88


In [56]:
next(iter(test_loader))[0][0]

tensor([[[-0.9934, -0.9934, -0.9934,  ..., -0.8868, -0.9018, -0.9256],
         [-0.9118, -0.9118, -0.8967,  ..., -0.9168, -0.9098, -0.9256],
         [-0.9118, -0.9118, -0.9118,  ..., -0.9118, -0.9018, -0.9256],
         ...,
         [ 3.1964,  2.7788,  2.6269,  ...,  2.8648,  3.0676,  2.7299],
         [ 3.1578,  3.1641,  2.9206,  ...,  2.8293,  2.7735,  2.5607],
         [-1.0342, -1.0342, -1.0342,  ...,  3.0777,  3.0169,  2.5697]],

        [[-1.2205, -1.2205, -1.2205,  ..., -1.1440, -1.1548, -1.1719],
         [-1.1620, -1.1620, -1.1512,  ..., -1.1656, -1.1606, -1.1719],
         [-1.1620, -1.1620, -1.1620,  ..., -1.1620, -1.1548, -1.1719],
         ...,
         [ 1.7850,  1.4854,  1.3765,  ...,  1.5471,  1.6926,  1.4503],
         [ 1.7573,  1.7618,  1.5871,  ...,  1.5217,  1.4817,  1.3290],
         [-1.2498, -1.2498, -1.2498,  ...,  1.6999,  1.6562,  1.3354]],

        [[-1.2494, -1.2494, -1.2494,  ..., -1.1724, -1.1833, -1.2004],
         [-1.1904, -1.1904, -1.1796,  ..., -1